In [361]:
import pandas as pd
import numpy as np
import random 
import os
import joblib
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import date
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


In [362]:
data = pd.read_csv("/Users/yj.noh/Desktop/total_working.csv", encoding = "utf-8")
data = data.sort_values(by=["brms_rider_id"])

print(data.shape) # 11,099

(11099, 28)


In [363]:
data = data.fillna('NA')

In [364]:
# 친구추천 데이터 삭제 
print(data['is_recom'].value_counts()) 
df = data

# df = data[data['is_recom'] == 0 ] 
# print(df.shape) #7,353

is_recom
0    7353
1    3746
Name: count, dtype: int64


In [365]:
# 날짜 형식 변환
df['first_available_date'] = pd.to_datetime(df['first_available_date'], format='%Y-%m-%d', errors='coerce')
df['business_start_day'] = pd.to_datetime(df['business_start_day'], format='%Y-%m-%d', errors='coerce')
df['last_working_day'] = pd.to_datetime(df['last_working_day'], format='%Y-%m-%d', errors='coerce')

## 1. first_available_date 없는 사람 / 첫 배달 가능일 존재, 실제 첫 배달 수행 여부 x 제외 

In [366]:
null_first_available_date_count = df['first_available_date'].isna().sum()
count_missing_business_start_day = df['first_available_date'].notnull() & df['business_start_day'].isnull()
count = df['last_working_day'].isna().sum()

print(f"'first_available_date'가 null인 데이터 개수: {null_first_available_date_count} 개") # 1,336
print(f"'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: {count_missing_business_start_day.sum()} 개") # 3,767
print({count}) #5114

'first_available_date'가 null인 데이터 개수: 1336 개
'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: 3767 개
{5114}


In [367]:
filtered_df = df[df['first_available_date'].notnull() & df['business_start_day'].notnull() & df['last_working_day'].notnull() ]
print(filtered_df.shape) # 5,985

(5985, 28)


## 2. 최소 한 건 이상 수행한 라이더 대상

In [368]:
print(filtered_df.isna().sum())

brms_rider_id                   0
birth                           0
delivery_method                 0
first_available_date            0
business_start_day              0
is_recom                        0
insurance_type                  0
gender                          0
recommender_rider_account_id    0
rider_no                        0
수행건수                            0
조회기간                            0
실제수행기간                          0
last_working_day                0
1순위수행시각                         0
2순위수행시각                         0
1순위평일/주말                        0
2순위평일/주말                        0
1순위수행요일                         0
2순위수행요일                         0
1순위기상타입                         0
주요수행service_type                0
주요수행allocated_type              0
주요수행method                      0
1순위수행전달거리                       0
2순위수행전달거리                       0
기본배달비_중간값                       0
총배달비_중간값                        0
dtype: int64


In [369]:
# 마지막 수행일 ~ 현재까지 기간
today = pd.to_datetime(date.today())

filtered_df['not_working'] = (today - filtered_df['last_working_day']).dt.days +1
filtered_df['첫운행이후'] = (today - filtered_df['business_start_day']).dt.days +1 

print(filtered_df.head(2))

                                          brms_rider_id  birth  \
5104  0003198AAE6FE85D1F86B2D4D45F992CFAE1D3E22256C0...   1990   
3618  001E1E700EBC869D19DB54B950E506910250998C3F0446...   1990   

        delivery_method first_available_date business_start_day  is_recom  \
5104               BIKE           2023-09-08         2023-09-14         0   
3618  THROTTLE_BICYCLES           2023-09-04         2023-09-08         0   

              insurance_type gender recommender_rider_account_id  \
5104  BIKE_ON_DEMAND_HYUNDAI    MAN                           NA   
3618           ETC_ON_DEMAND    MAN                           NA   

                                               rider_no  ... 1순위기상타입  \
5104  0003198AAE6FE85D1F86B2D4D45F992CFAE1D3E22256C0...  ...     0.0   
3618  001E1E700EBC869D19DB54B950E506910250998C3F0446...  ...     0.0   

     주요수행service_type 주요수행allocated_type         주요수행method 1순위수행전달거리  \
5104           SINGLE                0.0               BIKE    2000.0   
3

In [370]:
print(filtered_df['not_working'].value_counts())
print(filtered_df['첫운행이후'].value_counts())

not_working
2     1384
3      568
4      221
5      180
9      140
      ... 
68      12
67      10
71       7
70       6
72       2
Name: count, Length: 71, dtype: int64
첫운행이후
49    220
48    215
42    212
43    200
46    195
     ... 
9      13
15     13
7      12
6      12
5      11
Name: count, Length: 71, dtype: int64


In [371]:
def period (value):
    if value < 11:
        return '10일 이하'
    elif 11 <= value <= 20:
        return '20일 이하'
    elif 21 <= value <= 30 :
        return '30일 이하'
    elif 31 <= value <= 40 : 
        return '40일 이하'
    elif 41 <= value <= 50 :
        return '50일 이하'
    elif 51 <= value <= 60 :
        return '60일 이하'
    elif 61 <= value :
        return '61일 이상'
        
    
filtered_df['미운행기간'] = filtered_df['not_working'].apply(period)
filtered_df['첫운행이후기간'] = filtered_df['첫운행이후'].apply(period)

In [372]:
print(filtered_df['미운행기간'].value_counts())
print(filtered_df['첫운행이후기간'].value_counts())

미운행기간
10일 이하    2915
50일 이하     718
40일 이하     613
20일 이하     575
60일 이하     507
30일 이하     460
61일 이상     197
Name: count, dtype: int64
첫운행이후기간
50일 이하    1829
60일 이하    1586
61일 이상    1090
40일 이하     862
30일 이하     300
20일 이하     189
10일 이하     129
Name: count, dtype: int64


In [373]:
cross_table = pd.crosstab(filtered_df['미운행기간'], filtered_df['첫운행이후기간'])
print(cross_table)

첫운행이후기간  10일 이하  20일 이하  30일 이하  40일 이하  50일 이하  60일 이하  61일 이상
미운행기간                                                          
10일 이하      129     122     171     445     865     708     475
20일 이하        0      67      43      84     159     140      82
30일 이하        0       0      86      87     134      86      67
40일 이하        0       0       0     246     184     107      76
50일 이하        0       0       0       0     487     164      67
60일 이하        0       0       0       0       0     381     126
61일 이상        0       0       0       0       0       0     197


In [374]:
reference_date = pd.to_datetime('2023-10-10') - pd.DateOffset(days=21) #9/19 

filtered_df['outcome'] = (filtered_df['last_working_day'] < reference_date).astype(int)


In [375]:
filtered_df.to_csv('prj-ML-model-LT_OV30/on_boarding_data.csv', index= False, encoding = "utf-8")

## 첫 운행일 ~ 7일 동안의 배달건 

In [376]:
data_7 = pd.read_csv("/Users/yj.noh/Desktop/7day.csv", encoding = "utf-8")
data_7 = data_7.sort_values(by=["brms_rider_id"])

print(data_7.shape) # 11,099

(11099, 28)


In [377]:
data_7 = data_7.fillna('NA')

In [378]:
# 날짜 형식 변환
data_7['first_available_date'] = pd.to_datetime(data_7['first_available_date'], format='%Y-%m-%d', errors='coerce')
data_7['business_start_day'] = pd.to_datetime(data_7['business_start_day'], format='%Y-%m-%d', errors='coerce')
data_7['last_working_day'] = pd.to_datetime(data_7['last_working_day'], format='%Y-%m-%d', errors='coerce')

null_first_available_date_count = data_7['first_available_date'].isna().sum()
count_missing_business_start_day = data_7['first_available_date'].notnull() & data_7['business_start_day'].isnull()
count = data_7['last_working_day'].isna().sum()

print(f"'first_available_date'가 null인 데이터 개수: {null_first_available_date_count} 개") # 1,336
print(f"'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: {count_missing_business_start_day.sum()} 개") # 3,767
print({count}) #5114



'first_available_date'가 null인 데이터 개수: 1336 개
'first_available_date'는 있지만 'business_start_day'가 없는 데이터 개수: 3767 개
{5120}


In [379]:
df_7 = data_7[data_7['first_available_date'].notnull() & data_7['business_start_day'].notnull() & data_7['last_working_day'].notnull() ]
print(df_7.shape) # 5,985 

(5979, 28)


In [380]:
final_data = pd.merge(df_7, filtered_df[['brms_rider_id', 'outcome']], on = 'brms_rider_id', how='left')

print(final_data.shape) # 5985


(5979, 29)


In [381]:
final_data.to_csv('prj-ML-model-LT_OV30/on_boarding_data_7day.csv', index= False, encoding = "utf-8")